In [3]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry

gis = GIS("home")

/opt/conda/lib/python3.7/site-packages/arcgis/gis/__init__.py:430: UserWarning: You are logged on as apfister_EsriAidDev with an administrator role, proceed with caution.
  self.users.me.username)


In [4]:
states = ['TN']
field_name = 'STATE'

In [5]:
webapp_id = '10a4b73b893f4122852ed860697a89bc'
webmap_id = '96839c49f67b4b98b3147940cb9fbdb6'
feature_service_id = '6d223dcd23c547f78c60adec4a0d8f6c'
feature_service_id_in_webmap = 'The_Nature_Conservancy_Lands_Data_8734'

In [6]:
webapp = gis.content.get(webapp_id)
webmap = gis.content.get(webmap_id)
source_fl_item = gis.content.get(feature_service_id)
source_flc = FeatureLayerCollection.fromitem(source_fl_item)

### get and store extents for each state

In [7]:
print ('getting state extents ...')
fl = FeatureLayer.fromitem(gis.content.get('99fd67933e754a1181cc755146be21ca'))

fset = fl.query(where="1=1", out_fields='STATE_ABBR')
state_extent_cache = {}

for fs in fset:
       
    geom = Geometry(fs.geometry)
    
    ex = {
        'xmin': geom.extent[0],
        'ymin': geom.extent[1],
        'xmax': geom.extent[2],
        'ymax': geom.extent[3]
    }
    
    abbrev = fs.attributes['STATE_ABBR']
    
    state_extent_cache[abbrev] = ex
    
print ('done getting state extents')

getting state extents ...
done getting state extents


In [8]:
def get_state_extent(abbrev):
    return state_extent_cache[abbrev]

In [13]:
def update_webmap_state_props(wm, ex, tag):
    wmjson = wm.get_data(True)
    
    wmjson['initialState']['viewpoint']['targetGeometry']['xmin'] = ex['xmin']
    wmjson['initialState']['viewpoint']['targetGeometry']['ymin'] = ex['ymin']
    wmjson['initialState']['viewpoint']['targetGeometry']['xmax'] = ex['xmax']
    wmjson['initialState']['viewpoint']['targetGeometry']['ymax'] = ex['ymax']

    wm.update(item_properties={'tags': tag}, data=wmjson)

In [12]:
def update_wab_state_props(wab, ex, title, tag):
    wabjson = wab.get_data(True)
    
    if not 'mapOptions' in wabjson['map']:
        wabjson['map']['mapOptions'] = {}
        
    if not 'extent' in wabjson['map']['mapOptions']:
        wabjson['map']['mapOptions']['extent'] = {}
    
    wabjson['map']['mapOptions']['extent'] = {
        'xmin' : ex['xmin'],
        'ymin' : ex['ymin'],
        'xmax' : ex['xmax'],
        'ymax' : ex['ymax'],
        'spatialReference': {
            'wkid': 102100,
            'latestWkid': 3857
        }
    }
    
    wabjson['title'] = title

    wab.update(item_properties={'tags': tag}, data=wabjson)    

# Batch Create

In [14]:
for s in states:
    
    print ('==================================')
    print (f'starting clone process for {s}...')
    
    ## tag to apply to all items
    le_tag = 'TNCLandEdit'
    
    ## create new folder in My content for state
    print (f'creating folder for {s}...')
    folder_name = f'TNC Lands {s}'
    gis.content.create_folder(folder_name)
    
    ## create editing group for state
    print (f'creating editing Group for {s}...')
    group_name = f'TNC Lands {s} Editing Group'

    state_group = gis.groups.create(group_name,tags=f'tnc,editing, {s}, {le_tag}')
    
    ## create hosted fl view for state
    where_clause = f"{field_name} = '{s}'"

    ## create a hosted view for the state
    new_name = f'TNC Lands ({s}) Editing View'
    print (f'creating fl view for {s}...')
    new_view = source_flc.manager.create_view(name=new_name, capabilities='Create,Delete,Query,Update,Editing')
    new_view.update(item_properties={'tags': le_tag})

    view_flc = FeatureLayerCollection.fromitem(new_view)

    service_layer = view_flc.layers[0]

    print (f'updating fl view definition for {s}...')
    update_dict = {"viewDefinitionQuery" : where_clause}

    service_layer.manager.update_definition(update_dict)

    # share to state group
    print (f'sharing view for {s} to {state_group.title}...')
    new_view.share(groups=[state_group])

    # move to state folder
    new_view.move(folder_name)
    
    ## clone wab
    print (f'cloning web app builder for {s}...')
    item_mapping = {

        # don't clone base TNC Layer, use newly created view instead
        '6d223dcd23c547f78c60adec4a0d8f6c': new_view.id,

        # don't clone USA Generalized States layer, just use it instead
        '99fd67933e754a1181cc755146be21ca': '99fd67933e754a1181cc755146be21ca'
    }
    cloned_webapp = gis.content.clone_items(items=[webapp], copy_data=False, search_existing_items=False, folder=folder_name, item_mapping=item_mapping)
    
    ## rename items from clone; move to state folder; share with state group
    print (f'renaming, moving, & sharing web app builder cloned items for {s}...')
    for item in cloned_webapp:
        ex = get_state_extent(s)
        
        new_name = f'TNC Lands ({s}) Editing '
        if item.type == 'Web Map':
            new_name = f'{new_name} Web Map'            
            update_webmap_state_props(item, ex, le_tag)
        else:
            new_name = f'{new_name} Web App'
            update_wab_state_props(item, ex, new_name, le_tag)

        item.update(item_properties={'title': new_name }) 
        item.share(groups=[state_group])
    
    print (f'done for {s}.')
    
print('all done.')    

starting clone process for TN...
creating folder for TN...
creating editing Group for TN...
creating fl view for TN...


NameError: name 'tags' is not defined